# 如何使用？
要给Agent添加短期记忆（线程级持久化），在创建 agent 时需要指定一个 `checkpointer` 。
> LangChain 将短期记忆作为 Agent 的 State 状态的一部分进行管理。
>
> 通过将这些信息存储在 graph 的状态中，agent 能够在给定的对话中访问全部的上下文信息。与此同时，还可以在不同的线程中保持隔离性。
>
> State通过使用 `checkpointer`持久化到数据库（或内存）中，以便于线程可以在任何时间点被恢复，所以线程就可以在任何时间点访问之前的上下文信息。
>
> 短期记忆在 agent 被调用时或一个步骤（如工具调用）完成时发生一次变更，并且在每个步骤开始的时候，会读取State 状态。


代码示例：

In [4]:
import dotenv
import os

from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 创建带有内存检查点的 agent
agent = create_agent(
    "gpt-5-mini",
    # [get_user_info],
    checkpointer=InMemorySaver(),
)

# 第一次对话：用户介绍自己
response1 = agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},
)

print("第一次对话响应:")
print(response1["messages"][-1].content)

# 第二次对话：询问用户信息
response2 = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    {"configurable": {"thread_id": "1"}},
)

print("\n第二次对话响应:")
print(response2["messages"][-1].content)
#
# # 第三次对话：在同一个线程中继续对话
# response3 = agent.invoke(
#     {"messages": [{"role": "user", "content": "Can you remember what I told you before?"}]},
#     {"configurable": {"thread_id": "1"}},
# )
#
# print("\n第三次对话响应:")
# print(response3)
#
# # 新线程对话：验证线程隔离性
# response4 = agent.invoke(
#     {"messages": [{"role": "user", "content": "What's my name?"}]},
#     {"configurable": {"thread_id": "2"}},  # 使用不同的线程ID
# )
#
# print("\n新线程对话响应:")
# print(response4)


第一次对话响应:
Hi Bob — nice to meet you! How can I help you today?

第二次对话响应:
You said your name is Bob.
